In [8]:
#import findspark
#findspark.init()
import pyspark
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as func
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("COM6012 Assignment 1 Task2") \
    .getOrCreate()

sc = spark.sparkContext


In [9]:
lines = spark.read.text("ml-25m/ratings.csv").rdd
parts = lines.map(lambda row: row.value.split(","))

header = parts.first()
parts = parts.filter(lambda line: line != header)

In [10]:
for line in parts.take(5):
    print(line)

['1', '296', '5.0', '1147880044']
['1', '306', '3.5', '1147868817']
['1', '307', '5.0', '1147868828']
['1', '665', '5.0', '1147878820']
['1', '899', '3.5', '1147868510']


In [11]:
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)

In [12]:
(training, test) = ratings.randomSplit([0.8, 0.2])

In [13]:
als_1 = ALS(maxIter=10, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

In [16]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")

param_grid = ParamGridBuilder()\
            .addGrid(als_1.rank,[5])\
            .addGrid(als_1.maxIter,[10])\
            .addGrid(als_1.regParam,[0.1])\
            .build()

cv = CrossValidator(estimator = als_1,
                    evaluator = evaluator,
                    estimatorParamMaps =param_grid,
                    numFolds = 3)

model = cv.fit(training)

In [18]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse_1 = evaluator.evaluate(predictions)
evaluator = RegressionEvaluator(metricName="mae", labelCol="rating",predictionCol="prediction")
mae_1 = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse_1))
print("Mean Absolute Error  = " + str(mae_1))

Root-mean-square error = 0.8121090305349241
Mean Absolute Error  = 0.6239368055651031


In [ ]:
import math
tr_idfs= []
dev_idfs = []
N_tr_idfs = len(tr_ngram_list)
N_dev_idfs = len(dev_ngram_list)
for ngram in vocab:
    if ngram in df:
        idf = math.log10(N_tr_idfs/df[ngram])
    else:
        idf == 0
    tr_idfs.append(tr_idfs)

for ngram in vocab:
    if ngram in df:
        idf = math.log10(N_dev_idfs/df[ngram])
    else:
        idf == 0
    dev_idfs.append(dev_idfs)